In [12]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='rt8i2KCzpdnA9C0M3NOji7bDMMHqwam8YSiE5N6Zuw70',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'arryhthmiaclassification-donotdelete-pr-c9av13mtwzpxls'
object_key = 'ECG_Classification.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
if not hasattr(streaming_body_1,"__iter__"):streaming_body_1.__iter__ = types.MethodType(__iter__,streaming_body_1)

In [13]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
filepath = unzip.namelist()
for path in filepath:
    unzip.extract(path)

In [1]:
pwd

'/home/wsuser/work'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')
x_test=test_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/test',target_size=(64,64),batch_size=32,class_mode='categorical')

import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(32))
model.add(Dense(6,activation='softmax'))
model.summary()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))

model.save('ECG.h5')



Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 32)                984096    
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 985,190
Trainable param

In [16]:

import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model=load_model("ECG.h5")
print("Loading Model")
img=tf.keras.utils.load_img("/home/wsuser/work/data/test/Normal/fig_2114.png",target_size=(64,64))
x=tf.keras.utils.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred=model.predict(x)
print("Predicting...")
classes_x=np.argmax(pred,axis=1)
classes_x
#predict_x=model.predict(X_test)classes_x=np.argmax(predict_x,axis=1)
index=["Left Bundle Branch Block","Normal","Premature Atrial Contraction","Premature Ventricular Contractions","Right Bundle Branch Block","Ventricular Fibrillation"]
result=str(index[classes_x[0]])
result
print("You Have been detected as",result)


1/1 [==============================] - 0s 40ms/step


'Normal'